In [1]:
import sys
sys.path.append('/workspaces/quantum/')
from colors import Bcolors as bc

In [2]:
import random
import numpy as np

# Load the training data
with open('../data/split-data/train.npy', 'rb') as f:
    train_input = np.load(f)
    train_labels = np.load(f)

# Load the testing data
with open('../data/split-data/test.npy', 'rb') as f:
    test_input = np.load(f)
    test_labels = np.load(f)

In [3]:
# The classification runner
def run(f_classify, x):
    return list(map(f_classify, x))

In [4]:
# The classification evaluator
def evaluate(predictions, actual) -> str:
    correct = list(filter(
        lambda item: item[0] == item[1],
        list(zip(predictions, actual))
    ))
    
    return '{} correct predictions out of {}. Accuracy {:.0f} %' \
        .format(len(correct), len(actual), 100*len(correct)/len(actual))

In [5]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

def specificity(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[0][1]) if (matrix[0][0]+matrix[0][1] > 0) else 0

def npv(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[1][0]) if (matrix[0][0]+matrix[1][0] > 0) else 0

In [6]:
def print_classifier_scores(train_labels, predictions, cm):  
    print(evaluate(predictions, train_labels))
    print()
    print('The precision score is {:.2f}'
        .format(precision_score(train_labels, predictions)))
    print('The recall score is {:.2f}'
        .format(recall_score(train_labels, predictions)))
    print('The specificity score is {:.2f}'
        .format(specificity(cm)))
    print('The npv score is {:.2f}'
       .format(npv(cm)))

# 😈 Unmaksing the Hypocrite Classifier

Define a function for the hypocrite classifier.

In [7]:
def hypocrite(passenger, weight):
    return round(min(1,max(0,weight*0.5+random.uniform(0, 1))))

hypocrite_predictions = run(lambda passenger: hypocrite(passenger, -0.5), train_input)
hypocrite_cm = confusion_matrix(train_labels, hypocrite_predictions)

In [9]:
print(f'😈 {bc.HEADER}Hypocrite classifier:{bc.ENDC}')
print_classifier_scores(train_labels, hypocrite_predictions, hypocrite_cm)

😈 Hypocrite classifier:
369 correct predictions out of 711. Accuracy 52 %

The precision score is 0.34
The recall score is 0.26
The specificity score is 0.69
The npv score is 0.59
